In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import *
import tensorflow_probability as tfp


C:\Users\user\.conda\envs\Tensorflow\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\.conda\envs\Tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\.conda\envs\Tensorflow\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv('eurusd_h1_5000.csv')

In [3]:
df.pop("Unnamed: 0")
df.pop("time")
#df.pop("open")
#df.pop("high")
#df.pop("low")


0       2021-11-08 02:00:00
1       2021-11-08 03:00:00
2       2021-11-08 04:00:00
3       2021-11-08 05:00:00
4       2021-11-08 06:00:00
               ...         
4995    2022-08-25 10:00:00
4996    2022-08-25 11:00:00
4997    2022-08-25 12:00:00
4998    2022-08-25 13:00:00
4999    2022-08-25 14:00:00
Name: time, Length: 5000, dtype: object

In [4]:
df

,open,high,low,close
0,1.15680,1.15680,1.15582,1.15604
1,1.15604,1.15624,1.15539,1.15548
2,1.15548,1.15593,1.15532,1.15593
3,1.15593,1.15623,1.15574,1.15577
4,1.15578,1.15634,1.15566,1.15610
...,...,...,...,...
4995,1.00106,1.00237,0.99953,0.99996
4996,0.99994,1.00122,0.99871,1.00072
4997,1.00074,1.00113,0.99868,0.99972
4998,0.99972,0.99979,0.99741,0.99803


In [5]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data = transformed_dataset, index = df.index)

In [6]:
number_of_rows = df.values.shape[0]
window_length= 10 # the number of past games needed to make prediction
number_of_features = df.values.shape[1]

In [7]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype =float)
label = np.empty([number_of_rows-window_length, number_of_features], dtype = float)
window_length = 10

for i in range(0, number_of_rows-window_length):
    train[i] = transformed_df.iloc[i:i+window_length, 0: number_of_features]
    label[i] = transformed_df.iloc[i+window_length: i+window_length+1, 0: number_of_features]

In [165]:
model = tf.keras.Sequential([
  
#layers.experimental.preprocessing.Rescaling(1./255),
layers.LSTM(4),
layers.Dense(64, input_dim=8, activation=None),
layers.Dense(14, activation=None),
layers.Dense(4, activation=None),
])

#model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.1), loss='mae', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Nadam(), loss='mae', metrics=['accuracy'])

In [166]:
checkpoint = ModelCheckpoint("my_model_weights2.h5", monitor='val_accuracy', verbose=1,
    save_best_only=True, mode='auto', period=1)
early_stopping_monitor = EarlyStopping(patience=2)

i = 0
while i < 200:
    model.fit(train, 
            label,
          batch_size = 3, 
            epochs = 1,
            validation_split=0.1,
            callbacks=[checkpoint])
    #model.save_weights('my_model_weights2.h5')
    model.load_weights('my_model_weights2.h5')
    i+=1

1496/1497 [============================>.] - ETA: 0s - loss: 0.0555 - accuracy: 0.2870
Epoch 1: val_accuracy improved from -inf to 0.07816, saving model to my_model_weights2.h5
1497/1497 [==============================] - ETA: 0s - loss: 0.0270 - accuracy: 0.3042
Epoch 1: val_accuracy improved from 0.07816 to 0.24248, saving model to my_model_weights2.h5
1497/1497 [==============================] - ETA: 0s - loss: 0.0246 - accuracy: 0.3055
Epoch 1: val_accuracy improved from 0.24248 to 0.48096, saving model to my_model_weights2.h5
1493/1497 [============================>.] - ETA: 0s - loss: 0.0225 - accuracy: 0.3237
Epoch 1: val_accuracy did not improve from 0.48096
1493/1497 [============================>.] - ETA: 0s - loss: 0.0225 - accuracy: 0.3061
Epoch 1: val_accuracy did not improve from 0.48096
1495/1497 [============================>.] - ETA: 0s - loss: 0.0224 - accuracy: 0.3099
Epoch 1: val_accuracy improved from 0.48096 to 0.49699, saving model to my_model_weights2.h5
1493/14

1495/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3173
Epoch 1: val_accuracy did not improve from 0.49900
1492/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3177
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0176 - accuracy: 0.3140
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.3197
Epoch 1: val_accuracy did not improve from 0.49900
1496/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3231
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3266
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [============================>.] - ETA: 0s - loss: 0.0178 - accuracy: 0.3130
Epoch 1: val_accuracy did not improve from 0.49900
1496/1497 [=======================

1494/1497 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.3224
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3050
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0176 - accuracy: 0.3164
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0177 - accuracy: 0.3102
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.3264
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0174 - accuracy: 0.3291
Epoch 1: val_accuracy did not improve from 0.49900
1496/1497 [============================>.] - ETA: 0s - loss: 0.0174 - accuracy: 0.3184
Epoch 1: val_accuracy did not improve from 0.49900
1494/1497 [=======================

1497/1497 [==============================] - ETA: 0s - loss: 0.0177 - accuracy: 0.3184
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3103
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.3251
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.3164
Epoch 1: val_accuracy did not improve from 0.49900
1496/1497 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.3258
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0175 - accuracy: 0.3202
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0178 - accuracy: 0.3122
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [=======================

1495/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3151
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3264
Epoch 1: val_accuracy did not improve from 0.49900
1494/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3162
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.3193
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0176 - accuracy: 0.3164
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0176 - accuracy: 0.3262
Epoch 1: val_accuracy did not improve from 0.49900
1494/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3289
Epoch 1: val_accuracy did not improve from 0.49900
1494/1497 [=======================

1492/1497 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.3168
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3329
Epoch 1: val_accuracy did not improve from 0.49900
1492/1497 [============================>.] - ETA: 0s - loss: 0.0178 - accuracy: 0.3132
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.3251
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0178 - accuracy: 0.3184
Epoch 1: val_accuracy did not improve from 0.49900
1493/1497 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.3112
Epoch 1: val_accuracy did not improve from 0.49900
1494/1497 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.3184
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [=======================

1494/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3264
Epoch 1: val_accuracy did not improve from 0.49900
1497/1497 [==============================] - ETA: 0s - loss: 0.0178 - accuracy: 0.3111
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3168
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0174 - accuracy: 0.3162
Epoch 1: val_accuracy did not improve from 0.49900
1494/1497 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.3115
Epoch 1: val_accuracy did not improve from 0.49900
1496/1497 [============================>.] - ETA: 0s - loss: 0.0178 - accuracy: 0.3229
Epoch 1: val_accuracy did not improve from 0.49900
1495/1497 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.3059
Epoch 1: val_accuracy did not improve from 0.49900
1496/1497 [=======================

KeyboardInterrupt: 

In [167]:
model.evaluate(train, label, batch_size = 2)

2495/2495 [==============================] - 21s 8ms/step - loss: 0.0187 - accuracy: 0.3437


[0.018728548660874367, 0.34368738532066345]